In [1]:
# Data を読み込んで把握する
import pandas as pd

# Jim の利用履歴(2018/04 ~ 2019/03)の Data
uselog = pd.read_csv('sample_code/chapter_3/use_log.csv')
print(len(uselog))
uselog.head()

197428


,log_id,customer_id,usedate
0,L00000049012330,AS009373,2018-04-01
1,L00000049012331,AS015315,2018-04-01
2,L00000049012332,AS040841,2018-04-01
3,L00000049012333,AS046594,2018-04-01
4,L00000049012334,AS073285,2018-04-01


### uselog(利用履歴)
- 顧客ID、利用日を含んだ３列のみの Simple な Data.
- どの顧客がいつ Jim を利用したのかがわかる。
- 件数は、197,428件と縦に長い。

In [2]:
# 2019/03 末時点での会員 Data
customer = pd.read_csv('sample_code/chapter_3/customer_master.csv')
print(len(customer))
customer.head()

4192


,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0


### customer(会員)
- 顧客ID、名前、会員 Class、性別、登録日等の情報が含まれている。
- 名前は Masking されており、名前だけでは個人は特定できないようになっている。
- is_delete 列は、2019年3月時点で大会している User を System 的に素早く検索するための列
   => **uselog と customer_id の紐づけが可能。**
- すでい退会済みの User も含めて4,192人。
- class, campaign_id 列を用いれば、self data と結合できる。

In [3]:
# 会員区分 Data
class_master = pd.read_csv('sample_code/chapter_3/class_master.csv')
print(len(class_master))
class_master.head()

3


,class,class_name,price
0,C01,オールタイム,10500
1,C02,デイタイム,7500
2,C03,ナイト,6000


In [4]:
# Campaign 区分 Data
campaign_master = pd.read_csv('sample_code/chapter_3/campaign_master.csv')
print(len(campaign_master))
campaign_master.head()

3


,campaign_id,campaign_name
0,CA1,通常
1,CA2,入会費半額
2,CA3,入会費無料


### 加工にあたり、主にする Data
- customer(顧客 Data)
- uselog(利用履歴)

が考えらえる。

まず、Data 数の少ない customer(顧客 Data)を主に考えていく。

一旦、uselog(利用履歴) data を置いておいて customer(顧客 Data)を整形し、どのような顧客が何人くらいいるのか等の全体像を掴む。

In [5]:
# 顧客 Data を整形する

customer_join = pd.merge(customer, class_master, on='class', how='left')
customer_join = pd.merge(customer_join, campaign_master, on='campaign_id', how='left')
customer_join

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
0,OA832399,XXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
1,PL270116,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
2,OA974876,XXXXX,C01,M,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
3,HD024127,XXXXX,C01,F,2015-05-01 00:00:00,NaN,CA1,0,オールタイム,10500,通常
4,HD661448,XXXXX,C03,F,2015-05-01 00:00:00,NaN,CA1,0,ナイト,6000,通常
...,...,...,...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,M,2019-03-14 00:00:00,NaN,CA1,0,オールタイム,10500,通常
4188,HD246549,XXXXX,C01,F,2019-03-14 00:00:00,NaN,CA1,0,オールタイム,10500,通常
4189,GD037007,XXXXX,C03,M,2019-03-14 00:00:00,NaN,CA1,0,ナイト,6000,通常
4190,OA953150,XXXXX,C01,M,2019-03-14 00:00:00,NaN,CA1,0,オールタイム,10500,通常


In [6]:
print(len(customer))
print(len(customer_join))

4192
4192


会員区分や金額等がわかるように Data 整形完了。
Data 件数も、Join 前後で変化がないことを確認。

欠損値の確認をする。

In [7]:
customer_join.isnull().sum()

customer_id         0
name                0
class               0
gender              0
start_date          0
end_date         2842
campaign_id         0
is_deleted          0
class_name          0
price               0
campaign_name       0
dtype: int64

- end_date 以外は欠損値が0
- end_date に欠損値が入っていること以外は比較的綺麗な Data.
  退会していない User は退会日である end_date を保持していないため、欠損値となっていると思われる。

顧客 Data を集計し全体像を見ていく。
- 会員区分
- Campaign 区分
- 性別
- すでに退会済みかどうか（is_deleted列）

毎に全体数を把握する。

In [8]:
# 会員区分
customer_join.groupby('class_name').count()['customer_id']

class_name
オールタイム    2045
デイタイム     1019
ナイト       1128
Name: customer_id, dtype: int64

オールタイムが約半数を占め、次いでナイト、最後にデイタイムとなっている。

In [9]:
# Campaign 区分
customer_join.groupby('campaign_name').count()['customer_id']

campaign_name
入会費半額     650
入会費無料     492
通常       3050
Name: customer_id, dtype: int64

通常入会が多く、入会キャンペーン中に入会したのはおよそ20%となっている。

In [10]:
# 性別
customer_join.groupby('gender').count()['customer_id']

gender
F    1983
M    2209
Name: customer_id, dtype: int64

男女比はわずかに男性が多い。

In [11]:
# すでに退会済みかどうか
customer_join.groupby('is_deleted').count()['customer_id']

is_deleted
0    2842
1    1350
Name: customer_id, dtype: int64

退会している User は1,350人で2019年３月時点で入会している User は2,842人。

仮説や疑問点
- Campaign はいつ行われていたのか。
- 性別と会員 Class の関係
- 今年度の入会人数
- etc...

2018年４月１日から2019年３月31日までの入会人数を集計してみる。

In [13]:
customer_join['start_date'] = pd.to_datetime(customer_join['start_date'])
customer_start = customer_join.loc[customer_join['start_date'] > pd.to_datetime('20180401')]
print(len(customer_start))

1361


,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name
2831,HI195873,XXX,C01,F,2018-04-02,2018-08-31 00:00:00,CA3,1,オールタイム,10500,入会費無料
2832,TS322485,XX,C03,F,2018-04-02,2018-08-31 00:00:00,CA3,1,ナイト,6000,入会費無料
2833,IK552971,XXXXX,C02,F,2018-04-02,2018-08-31 00:00:00,CA3,1,デイタイム,7500,入会費無料
2834,PL000560,XXX,C01,M,2018-04-02,2018-07-31 00:00:00,CA3,1,オールタイム,10500,入会費無料
2835,HD218970,XXXXX,C01,F,2018-04-02,2018-11-30 00:00:00,CA3,1,オールタイム,10500,入会費無料
...,...,...,...,...,...,...,...,...,...,...,...
4187,HD676663,XXXX,C01,M,2019-03-14,NaN,CA1,0,オールタイム,10500,通常
4188,HD246549,XXXXX,C01,F,2019-03-14,NaN,CA1,0,オールタイム,10500,通常
4189,GD037007,XXXXX,C03,M,2019-03-14,NaN,CA1,0,ナイト,6000,通常
4190,OA953150,XXXXX,C01,M,2019-03-14,NaN,CA1,0,オールタイム,10500,通常


入会会員は1,361人で退会した人数よりも11人多いが、全体の会員数は変わらない結果となった。